In [1]:
#|hide
from fastrl.test_utils import initialize_notebook
initialize_notebook()

In [2]:
#|default_exp learner.core

In [3]:
#|export
# Python native modules
import os
from contextlib import contextmanager
from typing import List,Union,Dict,Optional,Iterable
# Third party libs
from fastcore.all import add_docs
import torchdata.datapipes as dp
from torchdata.dataloader2.graph import list_dps 
import torch
from torch import nn
from torchdata.dataloader2 import DataLoader2
from torchdata.dataloader2.graph import traverse_dps,DataPipeGraph,DataPipe
# Local modules
from fastrl.torch_core import evaluating
from fastrl.pipes.core import find_dp
from fastrl.loggers.core import Record,EpochCollector

# Learner Core
> Core DataPipes for building Learners

In [4]:
#|export
class LearnerBase(dp.iter.IterDataPipe):
    def __init__(self,
            # The base NN that we getting raw action values out of.
            # This can either be a `nn.Module` or a dict of multiple `nn.Module`s
            # For multimodel training
            model:Union[nn.Module,Dict[str,nn.Module]], 
            # The dataloaders to read data from for training. This can be a single
            # DataLoader2 or an iterable that yields from a DataLoader2.
            fit_dls:Union[DataLoader2,Iterable], 
            # The dataloaders to read data from for validation. This can be a single
            # DataLoader2 or an iterable that yields from a DataLoader2.
            val_dls:Optional[Union[DataLoader2,Iterable]]=None, 
            # By default for reinforcement learning, we want to keep the workers
            # alive so that simluations are not being shutdown / restarted.
            # Epochs are expected to be handled semantically via tracking the number 
            # of batches.
            infinite_dls:bool=True
    ):
        self.model = model
        self.fit_iterable = fit_dls
        self.val_iterable = val_dls
        self.learner_base = self
        self.infinite_dls = infinite_dls
        self._dls = None
        self._ended = False
        self._validating = False

    def __getstate__(self):
        state = {k:v for k,v in self.__dict__.items() if k not in ['_dls']}
        # TODO: Needs a better way to serialize / deserialize states.
        # state['iterable'] = [d.state_dict() for d in state['iterable']]
        if dp.iter.IterDataPipe.getstate_hook is not None:
            return dp.iter.IterDataPipe.getstate_hook(state)

        return state

    def __setstate__(self, state):
        # state['iterable'] = [d.from_state_dict() for d in state['iterable']]
        for k,v in state.items():
            setattr(self,k,v)

    def end(self):
        self._ended = True
   
    def __iter__(self):
        self._ended = False
        for data in (self.val_iterable if self._validating else self.fit_iterable):
            if self._ended:
                break
            yield data

add_docs(
LearnerBase,
"Combines models,dataloaders, and optimizers together for running a training pipeline.",
reset="""If `infinite_dls` is false, then all dls will be reset, otherwise they will be
kept alive.""",
end="When called, will cause the Learner to stop iterating and cleanup."
)

In [5]:
#|export
class LearnerHead(dp.iter.IterDataPipe):
    def __init__(self,source_datapipe):
        self.source_datapipe = source_datapipe
        self.learner_base = find_dp(traverse_dps(self.source_datapipe),LearnerBase)

    def __iter__(self): yield from self.source_datapipe
    
    def fit(self,epochs):
        epocher = find_dp(traverse_dps(self),EpochCollector)
        epocher.epochs = epochs
        
        for iteration in self: 
            pass

    def validate(self,epochs=1,show=True) -> DataPipe:
        with evaluating(self.learner_base.model):
            try:
                self.learner_base._validating = True
                epocher = find_dp(traverse_dps(self),EpochCollector)
                epocher.epochs = epochs
                for iteration in self: 
                    pass
            finally:
                self.learner_base._validating = False

            if show:
                pipes = list_dps(traverse_dps(self.learner_base.val_iterable))
                for pipe in pipes:
                    if hasattr(pipe,'show'):
                        return pipe.show() 
        
add_docs(
LearnerHead,
"""
""",
fit="Runs the `LearnerHead` pipeline for `epochs`",
validate="""If there is more than 1 dl, then run 1 epoch of that dl based on 
`dl_idx` and returns the original datapipe for displaying."""
)  

In [6]:
from fastrl.dataloading.core import dataloaders
from fastrl.loggers.core import EpochCollector

In [11]:
class Printer(dp.iter.IterDataPipe):
    def __init__(self,pipe): 
        self.pipe = pipe

    def __iter__(self):
        for o in self.pipe:
            print(o, end=" ")
            yield o
        print()

def TestLearner(dls,val_dls):
    learner = LearnerBase(nn.Module(),dls,val_dls=val_dls)
    learner = Printer(learner)
    learner = EpochCollector(learner)
    learner = LearnerHead(learner)
    return learner

dls = dataloaders((
        dp.iter.IterableWrapper(range(10)),
        dp.iter.IterableWrapper(range(10,20))
    ),
    do_concat=True
)
(dl3,) = dataloaders(dp.iter.IterableWrapper(range(20,30)))
print('Concated Dataloaders')

learn = TestLearner(dls,dl3)
learn.fit(5)

print("Validating Concated Dataloaders:")
learn.validate(1)  # using one epoch for validation by default

dls = dataloaders((
        dp.iter.IterableWrapper(range(10)),
        dp.iter.IterableWrapper(range(10,20))
    ),
    do_multiplex=True
)
(dl3,) = dataloaders(dp.iter.IterableWrapper(range(20,30)))
print('Muxed Dataloaders')

learn = TestLearner(dls,dl3)
learn.fit(5)

print("Validating Muxed Dataloaders:")
learn.validate(1)

Concated Dataloaders
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
Validating Concated Dataloaders:
20 21 22 23 24 25 26 27 28 29 
Muxed Dataloaders
0 10 1 11 2 12 3 13 4 14 5 15 6 16 7 17 8 18 9 19 
0 10 1 11 2 12 3 13 4 14 5 15 6 16 7 17 8 18 9 19 
0 10 1 11 2 12 3 13 4 14 5 15 6 16 7 17 8 18 9 19 
0 10 1 11 2 12 3 13 4 14 5 15 6 16 7 17 8 18 9 19 
0 10 1 11 2 12 3 13 4 14 5 15 6 16 7 17 8 18 9 19 
Validating Muxed Dataloaders:
20 21 22 23 24 25 26 27 28 29 


> Warning: Pickling the LearnerBase will exclude the '_dls','opt','iterable' fields since
these aren't easily picklable (yet).

In [12]:
from fastrl.agents.dqn.basic import DQN
from fastrl.agents.core import AgentBase,AgentHead,StepFieldSelector,SimpleModelRunner,NumpyConverter
from fastrl.agents.discrete import ArgMaxer,PyPrimativeConverter
from fastrl.envs.gym import GymDataPipe

In [13]:
# Setup up the core NN
torch.manual_seed(0)
model = DQN(4,2)
# Setup the agent
agent = AgentBase(model,[])
agent = StepFieldSelector(agent,field='state')
# All the things that make this agent unique and special
# In this instance, all this module does is pass the action directly through to the model.
agent = SimpleModelRunner(agent)
agent = ArgMaxer(agent,only_idx=True)
agent = NumpyConverter(agent)
agent = PyPrimativeConverter(agent)
# Bring everything together into the AgentHead where actions will be passed and then run through the pipeline
agent = AgentHead(agent)

In [15]:
from fastrl.loggers.vscode_visualizers import VSCodeDataPipe

In [16]:
# Setup the DataBlock
def gym_block(num_workers=0,vis=False):
    pipe = GymDataPipe(['CartPole-v1']*1,agent=agent,nsteps=1,nskips=1,firstlast=False,include_images=True,n=100,bs=1)
    if vis:
        pipe = VSCodeDataPipe(pipe)
    return pipe

dls = dataloaders((gym_block(),gym_block(vis=True)))


In [25]:
def TestLearner(model,dls):
    learner = LearnerBase(model,dls[0],dls[1])
    learner = EpochCollector(learner)
    learner = LearnerHead(learner)
    return learner

In [26]:
import pickle

In [27]:
learner = TestLearner(model,dls)

out = pickle.dumps(learner)
pickle.loads(out)

LearnerHead

In [33]:
learner.fit(2)

In [34]:
learner.validate(1)

In [29]:
#|export
class StepBatcher(dp.iter.IterDataPipe):
    def __init__(self,
            source_datapipe,
            device=None
        ):
        self.source_datapipe = source_datapipe
        self.device = device
        
    def vstack_by_fld(self,batch,fld):
        try:
            if self.device is None: return torch.vstack(tuple(getattr(step,fld) for step in batch))
            return torch.vstack(tuple(getattr(step,fld) for step in batch)).to(torch.device(self.device))
        except RuntimeError as e:
            print(f'Failed to stack {fld} given batch: {batch}')
            raise
        
    def __iter__(self):
        for batch in self.source_datapipe:
            cls = batch[0].__class__
            yield cls(**{fld:self.vstack_by_fld(batch,fld) for fld in cls._fields})

add_docs(
StepBatcher,
"Converts multiple `StepType` into a single `StepType` with the fields concated.",
vstack_by_fld="vstacks a `fld` in `batch`"
)

In [36]:
#|hide
#|eval: false
!nbdev_export

/usr/local/lib/python3.8/dist-packages/nbdev/export.py:54: UserWarning: Notebook '/home/fastrl_user/fastrl/nbs/07_Agents/02_Continuous/12u_agents.ppo.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
